##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 텍스트 로드하기

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

이 튜토리얼은 텍스트를 로드하고 전처리하는 두 가지 방법을 보여 줍니다.

- 먼저 Keras 유틸리티와 전처리 레이어를 사용합니다. 여기에는 데이터 표준화, 토큰화 및 벡터화를 위해 데이터를 `tf.data.Dataset`와 `tf.keras.layers.TextVectorization`으로 변환하는 `tf.keras.utils.text_dataset_from_directory`가 포함되어 있습니다. TensorFlow가 처음이라면 여기서부터 시작해야 합니다.
- 그런 다음 `tf.data.TextLineDataset`와 같은 하위 수준 유틸리티를 사용하여 텍스트 파일을 로드하고, 보다 세밀한 제어를 위해 `text.UnicodeScriptTokenizer` 및 `text.case_fold_utf8`과 같은 [TensorFlow Text](https://www.tensorflow.org/text) API를 사용하여 데이터를 처리합니다.

In [ ]:
!pip install "tensorflow-text==2.8.*"

In [ ]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

## 예제 1: 스택 오버플로 질문에 대한 태그 예측하기

첫 번째 예제로 스택 오버플로에서 프로그래밍 질문 데이터세트를 다운로드합니다. 각 질문(*"값으로 사전을 어떻게 정렬하나요?"*)마다 정확히 하나의 태그(`Python`, `CSharp`, `JavaScript`, 또는 `Java`)가 레이블로 지정됩니다. 여러분의 작업은 질문에 대한 태그를 예측하는 모델을 개발하는 것입니다. 이것은 중요하고 널리 적용 가능한 머신러닝 문제인 다중 클래스 분류 예제입니다.

### 데이터세트 다운로드 및 탐색하기

`tf.keras.utils.get_file`을 사용하여 스택 오버플로 데이터세트를 다운로드하고, 디렉터리 구조를 탐색하여 시작하도록 합니다.

In [ ]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent

In [ ]:
list(dataset_dir.iterdir())

In [ ]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

`train/csharp`, `train/java`, `train/python` 및 `train/javascript` 디렉터리에는 많은 텍스트 파일이 포함되어 있으며, 각 텍스트 파일의 내용은 스택 오버플로 질문입니다.

다음과 같이 예제 파일을 출력하고 데이터를 검사합니다.

In [ ]:
sample_file = train_dir/'python/1755.txt'

with open(sample_file) as f:
  print(f.read())

### 데이터세트 로드하기

다음으로 디스크로부터 데이터를 로드하고 데이터를 훈련에 적합한 형식으로 준비합니다. 이렇게 하기 위해 여러분은 `tf.keras.utils.text_dataset_from_directory` 유틸리티를 사용하여 레이블이 지정된 `tf.data.Dataset`를 생성합니다. `tf.data`는 입력 파이프라인을 구축하는 강력한 도구 모음입니다([tf.data: TensorFlow 입력 파이프라인 빌드](../../guide/data.ipynb) 가이드에서 자세히 알아보세요).

`tf.keras.utils.text_dataset_from_directory` API는 다음과 같은 디렉터리 구조를 예상합니다.

```
train/
...csharp/
......1.txt
......2.txt
...java/
......1.txt
......2.txt
...javascript/
......1.txt
......2.txt
...python/
......1.txt
......2.txt
```

머신러닝 실험을 실행할 때 데이터세트를 [훈련](https://developers.google.com/machine-learning/glossary#training_set), [검증](https://developers.google.com/machine-learning/glossary#validation_set) 및 [테스트](https://developers.google.com/machine-learning/glossary#test-set)의 세 부분으로 나누는 것이 가장 좋습니다.

스택 오버플로 데이터세트는 이미 훈련 세트와 테스트 세트로 나누어져 있지만 여기에는 검증 세트가 없습니다.

`validation_split`이 `0.2`(즉, 20%)로 설정된 `tf.keras.utils.text_dataset_from_directory`를 사용하여 훈련 데이터를 80:20의 비율로 분할하는 검증 세트를 생성합니다.

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

이전 셀 출력을 통해 알 수 있듯이 훈련 폴더에는 8,000개의 예제가 있으며 여러분은 그 중 80%(또는 6,400개의 예제)를 훈련에 사용할 것입니다. 여러분은 `tf.data.Dataset`를 `Model.fit`에 직접 전달하여 모델을 훈련할 수 있다는 것을 곧 배우게 될 것입니다.

먼저 데이터세트를 반복하고 몇 가지 예제를 인쇄하여 데이터에 대한 감각을 익히세요.

참고: 분류 문제의 난이도를 높이기 위해 데이터세트 작성자는 프로그래밍 질문에서 *Python*, *CSharp*, *JavaScript* 또는 *Java*라는 단어를 *blank*로 대체했습니다.

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Question: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

레이블은 `0`, `1`, `2` 또는 `3`입니다. 이들이 어떠한 문자열 레이블에 해당하는지 확인하려면 데이터세트의 `class_names` 속성을 검사하면 됩니다.


In [ ]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

다음으로 `tf.keras.utils.text_dataset_from_directory`를 사용하여 검증 및 테스트 세트를 만듭니다. 검증을 위해 훈련 세트의 나머지 1,600개 리뷰를 사용합니다.

참고: `tf.keras.utils.text_dataset_from_directory`의 `validation_split` 및 `subset` 인수를 사용할 때 검증 및 훈련 분할이 겹치지 않도록 임의 시드를 지정하거나 `shuffle=False`를 전달하도록 합니다.

In [ ]:
# Create a validation set.
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

In [ ]:
test_dir = dataset_dir/'test'

# Create a test set.
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

### 훈련을 위한 데이터세트 준비하기

다음으로 `tf.keras.layers.TextVectorization` 레이어를 사용하여 데이터를 표준화, 토큰화 및 벡터화합니다.

- *표준화*는 일반적으로 데이터세트를 단순화하기 위해 구두점이나 HTML 요소를 제거하도록 텍스트를 전처리하는 것을 일컫습니다.
- *토큰화*는 문자열을 토큰으로 분할하는 것을 일컫습니다(예: 문장을 공백을 사용하여 개별 단어로 분할).
- *벡터화*는 신경망에 제공할 수 있도록 토큰을 숫자로 변환하는 것을 일컫습니다.

위의 모든 작업은 이 레이어로 수행할 수 있습니다(`tf.keras.layers.TextVectorization` API 문서에서 각 작업에 대해 자세히 알아볼 수 있습니다).

참고 사항:

- 기본 표준화는 텍스트를 소문자로 변환하고 구두점을 제거합니다(`standardize='lower_and_strip_punctuation'`).
- 기본 토큰화는 공백으로 분할합니다(`split='whitespace'`).
- 기본 벡터화 모드는 `'int'`(`output_mode='int'`)입니다. 이 모드는 정수 인덱스를 출력합니다(토큰당 하나). 이 모드는 단어 순서를 고려하는 모델을 빌드하는 데 사용할 수 있습니다. `'binary'`와 같은 다른 모드를 사용하여 [bag-of-words](https://developers.google.com/machine-learning/glossary#bag-of-words) 모델을 빌드할 수도 있습니다.

`TextVectorization`을 사용하는 표준화, 토큰화 및 벡터화에 대해 자세히 알아보기 위해 다음 두 가지 모델을 빌드합니다.

- 먼저 `'binary'` 벡터화 모드를 사용하여 bag-of-words 모델을 빌드합니다.
- 그런 다음 1D ConvNet에서 `'int'` 모드를 사용합니다.

In [ ]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

`'int'` 모드의 경우 최대 어휘 크기 외에 명시적인 최대 시퀀스 길이(`MAX_SEQUENCE_LENGTH`)를 설정해야 레이어가 패딩되거나 시퀀스를 정확히 `output_sequence_length` 값으로 자릅니다.

In [ ]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

다음으로 전처리 레이어의 상태를 데이터세트에 맞추기 위해 `TextVectorization.adapt`를 호출합니다. 그러면 모델이 문자열 인덱스를 정수로 빌드합니다.

참고: 테스트세트를 사용하면 정보가 누출되므로 `TextVectorization.adapt`를 호출할 때 훈련 데이터만 사용하는 것이 중요합니다.

In [ ]:
# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

이러한 레이어를 사용하여 데이터를 전처리한 결과를 인쇄합니다.

In [ ]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [ ]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [ ]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

In [ ]:
print("'binary' vectorized question:",
      binary_vectorize_text(first_question, first_label)[0])

In [ ]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

위에 표시된 것처럼 `TextVectorization`의 `'binary'` 모드는 입력에 한 번 이상 존재하는 토큰을 나타내는 배열을 반환하는 반면 `'int'` 모드는 각 토큰을 정수로 대체하기에 원래 순서를 유지합니다.

레이어에서 `TextVectorization.get_vocabulary`를 호출하여 각 정수가 해당하는 토큰(문자열)을 조회할 수 있습니다.

In [ ]:
print("1289 ---> ", int_vectorize_layer.get_vocabulary()[1289])
print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

모델을 훈련할 준비가 거의 되었습니다.

최종 전처리 단계로 이전에 생성한 `TextVectorization` 레이어를 훈련, 검증 및 테스트 세트에 적용합니다.

In [ ]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

### 성능을 높이도록 데이터세트 구성하기

다음은 I/O가 차단되지 않도록 데이터를 로드할 때 사용해야 하는 두 가지 중요한 메서드입니다.

- `Dataset.cache`는 데이터가 디스크에서 로드된 후 메모리에 데이터를 보관합니다. 이렇게 하면 모델을 훈련하는 동안 데이터세트로 인한 병목 현상이 발생하지 않습니다. 데이터세트가 너무 커서 메모리에 맞지 않는 경우 이 메서드를 사용하여 성능이 뛰어난 온 디스크 캐시를 생성할 수도 있습니다. 다수의 작은 파일보다 읽기가 더 효율적입니다.
- `Dataset.prefetch`는 훈련하는 동안 데이터 전처리 및 모델 실행을 중첩시킵니다.

[tf.data API를 통한 성능 향상](../../guide/data_performance.ipynb) 가이드의 *프리페칭* 섹션에서 두 가지 메서드와 데이터를 디스크에 캐시하는 방법에 대해 자세히 알아볼 수 있습니다.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

### 모델 훈련하기

이제 신경망을 만들 차례입니다.

`'binary'` 벡터화된 데이터의 경우 간단한 bag-of-words 선형 모델을 정의한 다음 데이터를 구성하고 훈련합니다.

In [ ]:
binary_model = tf.keras.Sequential([layers.Dense(4)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

그런 다음 `'int'` 벡터화된 레이어를 사용하여 1D ConvNet을 빌드합니다.

In [ ]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [ ]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

두 모델을 비교합니다.

In [ ]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

In [ ]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

테스트 데이터에서 두 모델을 평가합니다.

In [ ]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

참고: 이 예제 데이터세트는 다소 단순한 분류 문제를 나타냅니다. 더 복잡한 데이터세트와 문제는 전처리 전략과 모델 아키텍처에서 미묘하지만 중요한 차이를 나타냅니다. 다양한 접근 방식을 비교하려면 다양한 하이퍼 매개 변수와 epochs를 시도해야 합니다.

### 모델 내보내기

위의 코드에서는 모델에 텍스트를 제공하기 전에 `tf.keras.layers.TextVectorization`을 데이터세트에 적용했습니다. 모델이 원시 문자열을 처리할 수 있도록 하려면(예: 배포를 단순화하기 위해) 모델 내부에 `TextVectorization` 레이어를 포함할 수 있습니다.

이를 위해 방금 훈련한 가중치를 사용하여 새 모델을 만들 수 있습니다.

In [ ]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))

이제 여러분의 모델은 원시 문자열을 입력으로 사용하고 `Model.predict`를 사용하여 각 레이블의 점수를 예측할 수 있습니다. 다음과 같이 최대 점수를 가진 레이블을 찾는 함수를 정의합니다.

In [ ]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

### 새 데이터에 대한 추론 실행하기

In [ ]:
inputs = [
    "how do I extract keys from a dict into a list?",  # 'python'
    "debug public static void main(string[] args) {...}",  # 'java'
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

모델 내부에 텍스트 전처리 논리를 포함하면 배포를 단순화하고 [훈련/테스트 왜곡](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew) 가능성을 줄이는 프로덕션용 모델을 내보낼 수 있습니다.

`tf.keras.layers.TextVectorization`를 적용할 위치를 선택할 때 염두에 두어야 할 성능 차이가 있습니다. 레이어를 모델 외부에서 사용하면 GPU에서 훈련할 때 비동기 CPU 처리 및 데이터 버퍼링을 수행할 수 있습니다. 따라서 GPU에서 모델을 훈련하는 경우 모델을 개발하는 동안 최상의 성능을 얻기 위해 이 옵션을 사용하고 배포 준비가 완료되면 모델 내부에 `TextVectorization` 레이어를 포함하도록 전환할 수 있습니다.

모델 저장에 대해 자세히 알아보려면 [모델 저장과 복원](../keras/save_and_load.ipynb) 튜토리얼을 방문하세요.

## 예제 2: 일리아드(Iliad) 번역의 작성자 예측하기


다음은 `tf.data.TextLineDataset`를 사용하여 텍스트 파일로부터 예제를 로드하고 [TensorFlow Text](https://www.tensorflow.org/text)를 사용하여 데이터를 전처리하는 예를 제공합니다. 호머의 일리아드 작품을 다르게 번역한 3개의 영어 번역문을 사용하게 되며, 한 줄의 텍스트가 제공되었을 때 번역가를 식별하는 모델을 훈련합니다.

### 데이터세트 다운로드 및 탐색하기

3가지 번역본은 다음과 같습니다.

- [William Cowper](https://en.wikipedia.org/wiki/William_Cowper): [텍스트](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)
- [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby): [텍스트](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)
- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29): [텍스트](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

이 튜토리얼에서 사용된 텍스트 파일은 문서 헤더와 바닥 글, 줄 번호 및 챕터 제목 등을 제거하는 일반적인 전처리 작업을 거쳤습니다.

이 가볍게 손질한 파일을 로컬로 다운로드합니다.

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = utils.get_file(name, origin=DIRECTORY_URL + name)

parent_dir = pathlib.Path(text_dir).parent
list(parent_dir.iterdir())

### 데이터세트 로드하기

이전에는 `tf.keras.utils.text_dataset_from_directory`를 사용할 경우 파일의 모든 콘텐츠를 단일 예제로 취급했습니다. 여기에서는 텍스트 파일로부터 `tf.data.Dataset`를 생성하도록 설계된 `tf.data.TextLineDataset`을 사용합니다. 이때 각 예제는 원본 파일의 텍스트 줄입니다. `TextLineDataset`은 주로 줄 기반의 텍스트 데이터(예: 시 또는 오류 로그)에 유용합니다.

이러한 파일을 반복하여 각 파일을 자체 데이터세트에 로드합니다. 각 예제는 개별적으로 레이블을 지정해야 하므로 `Dataset.map`을 사용하여 각 예제에 labeler 함수를 적용합니다. 이렇게 하면 데이터세트의 모든 예제를 반복하여 (`example, label`) 쌍을 반환합니다.

In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

In [ ]:
labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(str(parent_dir/file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

다음으로 `Dataset.concatenate`를 사용하여 레이블이 지정된 데이터세트를 단일 데이터세트로 결합한 후 `Dataset.shuffle`을 사용하여 셔플합니다.


In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

이전과 같이 몇 가지 예제를 출력합니다. 데이터세트가 아직 일괄 처리되지 않았으므로 `all_labeled_data`의 각 항목은 하나의 데이터 포인트에 해당합니다.

In [ ]:
for text, label in all_labeled_data.take(10):
  print("Sentence: ", text.numpy())
  print("Label:", label.numpy())

### 훈련을 위한 데이터세트 준비하기

`tf.keras.layers.TextVectorization`을 사용하여 텍스트 데이터세트를 전처리하는 대신에 이제는 TensorFlow Text API를 사용하여 데이터를 표준화 및 토큰화하고 어휘를 빌드하고, `tf.lookup.StaticVocabularyTable`을 사용하여 토큰을 모델에 정수에 매핑한 후 모델에 공급합니다([TensorFlow 텍스트](https://www.tensorflow.org/text)에 대해 자세히 알아보기).

다음과 같이 텍스트를 소문자로 변환하고 토큰화하는 함수를 정의합니다.

- TensorFlow Text는 다양한 토크나이저를 제공합니다. 이 예제에서는 `text.UnicodeScriptTokenizer`를 사용하여 데이터세트를 토큰화합니다.
- 여러분은 `Dataset.map`을 사용하여 데이터세트에 토큰화를 적용합니다.

In [ ]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [ ]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [ ]:
tokenized_ds = all_labeled_data.map(tokenize)

데이터세트를 반복하고 몇 가지 토큰화된 예제를 출력할 수 있습니다.


In [ ]:
for text_batch in tokenized_ds.take(5):
  print("Tokens: ", text_batch.numpy())

다음으로 빈도별로 토큰을 정렬하고 상위 `VOCAB_SIZE` 토큰을 유지하여 어휘를 빌드합니다.

In [ ]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

토큰을 정수로 변환하려면 `vocab` 세트를 사용하여 `tf.lookup.StaticVocabularyTable`을 생성합니다. [`2`, `vocab_size + 2`] 범위의 정수에 토큰을 매핑합니다. `TextVectorization` 레이어와 마찬가지로 `0`은 패딩을 나타내기 위해 예약되어 있으며 `1`은 OOV(out-of-vocabulary) 토큰을 나타내기 위해 예약되어 있습니다.

In [ ]:
keys = vocab
values = range(2, len(vocab) + 2)  # Reserve `0` for padding, `1` for OOV tokens.

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

마지막으로 토크나이저 및 조회 테이블을 사용하여 데이터세트를 표준화, 토큰화 및 벡터화하는 함수를 정의합니다.

In [ ]:
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

결과를 출력하기 위해 단일 예제에서 다음을 시도할 수 있습니다.

In [ ]:
example_text, example_label = next(iter(all_labeled_data))
print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

이제 `Dataset.map`을 사용하여 데이터세트에서 전처리 함수를 실행합니다.

In [ ]:
all_encoded_data = all_labeled_data.map(preprocess_text)

### 데이터세트를 훈련 및 검증 세트로 분할하기


Keras `TextVectorization` 레이어는 벡터화된 데이터도 일괄 처리하고 패딩합니다. 배치 내부의 예제는 크기와 모양이 같아야 하기 때문에 패딩이 필요하지만 이러한 데이터세트의 예제는 모두 같은 크기가 아니며 각 텍스트 줄의 단어 수도 다릅니다.

`tf.data.Dataset`은 데이터세트 분할 및 패딩 일괄 처리를 지원합니다.

In [ ]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [ ]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

이제 `validation_data` 및 `train_data`는 (`example, label`) 쌍의 모음이 아니라 배치의 모음입니다. 각 배치는 배열로 표시되는 한 쌍의 (*많은 예제*, *많은 레이블*)입니다.

이는 다음과 같습니다.

In [ ]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

패딩에는 `0`을 사용하고 OOV(out-of-vocabulary) 토큰에는 `1`을 사용하였기에 어휘 크기가 2배 증가했습니다.

In [ ]:
vocab_size += 2

이전과 같은 더 나은 성능을 위한 데이터세트를 구성합니다.

In [ ]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

### 모델 훈련하기

이전과 같이 이 데이터세트에서 모델을 훈련할 수 있습니다.

In [ ]:
model = create_model(vocab_size=vocab_size, num_labels=3)

model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

history = model.fit(train_data, validation_data=validation_data, epochs=3)

In [ ]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

### 모델 내보내기

원시 문자열을 입력으로 사용할 수 있는 모델을 만들기 위해 사용자 정의 전처리 함수와 동일한 단계를 수행하는 Keras `TextVectorization` 레이어를 생성하게 됩니다. 이미 어휘를 훈련했으므로 `TextVectorization.adapt` 대신 `TextVectorization.set_vocabulary`를 사용하여 새 어휘를 훈련할 수 있습니다.

In [ ]:
preprocess_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

preprocess_layer.set_vocabulary(vocab)

In [ ]:
export_model = tf.keras.Sequential(
    [preprocess_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
# Create a test dataset of raw strings.
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)

loss, accuracy = export_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

인코딩된 검증 세트의 모델과 원시 검증 세트에 대해 내보내기를 수행한 모델의 손실 및 정확성은 예상대로 동일합니다.

### 새 데이터에 대한 추론 실행하기

In [ ]:
inputs = [
    "Join'd to th' Ionians with their flowing robes,",  # Label: 1
    "the allies, and his armour flashed about him so that he seemed to all",  # Label: 2
    "And with loud clangor of his arms he fell.",  # Label: 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.argmax(predicted_scores, axis=1)

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

## TensorFlow 데이터세트(TFDS)를 사용하여 더 많은 데이터세트 다운로드하기


[TensorFlow 데이터세트](https://www.tensorflow.org/datasets/catalog/overview)로부터 더 많은 데이터세트를 다운로드할 수 있습니다.

이 예제에서는 [IMDB 대형 영화 리뷰 데이터세트](https://www.tensorflow.org/datasets/catalog/imdb_reviews)를 사용하여 감정 분류용 모델을 훈련합니다.

In [ ]:
# Training set.
train_ds = tfds.load(
    'imdb_reviews',
    split='train[:80%]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)

In [ ]:
# Validation set.
val_ds = tfds.load(
    'imdb_reviews',
    split='train[80%:]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)

몇 가지 예제를 출력합니다.

In [ ]:
for review_batch, label_batch in val_ds.take(1):
  for i in range(5):
    print("Review: ", review_batch[i].numpy())
    print("Label: ", label_batch[i].numpy())

이제 이전과 같이 데이터를 전처리하고 모델을 훈련할 수 있습니다.

참고: 이진 분류 문제이므로 여러분의 모델에 `tf.keras.losses.SparseCategoricalCrossentropy` 대신 `tf.keras.losses.BinaryCrossentropy`를 사용합니다.

### 훈련을 위한 데이터세트 준비하기

In [ ]:
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = train_ds.map(lambda text, labels: text)
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [ ]:
# Configure datasets for performance as before.
train_ds = configure_dataset(train_ds)
val_ds = configure_dataset(val_ds)

### 모델 생성, 구성 및 훈련하기

In [ ]:
model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=1)
model.summary()

In [ ]:
model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

In [ ]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

### 모델 내보내기

In [ ]:
export_model = tf.keras.Sequential(
    [vectorize_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
# 0 --> negative review
# 1 --> positive review
inputs = [
    "This is a fantastic movie.",
    "This is a bad movie.",
    "This movie was so bad that it was good.",
    "I will never say yes to watching this movie.",
]

predicted_scores = export_model.predict(inputs)
predicted_labels = [int(round(x[0])) for x in predicted_scores]

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label)

## 결론

이 튜토리얼에서는 텍스트를 로드하고 전처리하는 여러 방법을 보여 드렸습니다. 다음 단계로 다음과 같은 추가 텍스트 전처리 [TensorFlow 텍스트](https://www.tensorflow.org/text) 튜토리얼을 탐색할 수 있습니다.

- [TF 텍스트를 사용한 BERT 전처리](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)
- [TF 텍스트로 토큰화](https://www.tensorflow.org/text/guide/tokenizers)
- [하위 단어 토크나이저](https://www.tensorflow.org/text/guide/subwords_tokenizer)

[TensorFlow 데이터세트](https://www.tensorflow.org/datasets/catalog/overview)에서 새 데이터세트를 찾을 수도 있습니다. 그리고 `tf.data`에 대해 자세히 알아보려면 [입력 파이프라인 빌드](../../guide/data.ipynb) 가이드를 확인하세요.